In [6]:
from collections import deque
import gymnasium as gym
from gymnasium import spaces
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Topology():
    def __init__(self, n, model="random", density=1):
        self.n = n
        self.model = model
        self.density = density
        self.adjacency_matrix = self.make_adjacency_matrix()
        
    def make_adjacency_matrix(self) -> np.ndarray:
        """Make adjacency matrix of a clique network.
        Args:
            n (int): Number of nodes.
            density (float): Density of the clique network.

        Returns:
            np.ndarray: Adjacency matrix.
        """
        if self.density < 0 or self.density > 1:
            raise ValueError("Density must be between 0 and 1.")

        n_edges = int(self.n * (self.n - 1) / 2 * self.density)
        adjacency_matrix = np.zeros((self.n, self.n))

        if self.model == "dumbbell":
            adjacency_matrix[0, self.n-1] = 1
            adjacency_matrix[self.n-1, 0] = 1
            for i in range(1, self.n//2):
                adjacency_matrix[0, i] = 1
                adjacency_matrix[i, 0] = 1
            for i in range(self.n//2+1, self.n):
                adjacency_matrix[i-1, self.n-1] = 1
                adjacency_matrix[self.n-1, i-1] = 1
        elif self.model == "linear":
            for i in range(1, self.n):
                adjacency_matrix[i-1, i] = 1
                adjacency_matrix[i, i-1] = 1
        elif self.model == "random":
            for i in range(1, self.n):
                adjacency_matrix[i-1, i] = 1
                adjacency_matrix[i, i-1] = 1
                n_edges -= 1
            # If the density of the current adjacency matrix is over density, return it.
            if n_edges <= 0:
                return adjacency_matrix
            else:
                arr = [1]*n_edges + [0]*((self.n-1)*(self.n-2)//2 - n_edges)
                np.random.shuffle(arr)
                for i in range(0, self.n):
                    for j in range(i+2, self.n):
                        adjacency_matrix[i, j] = arr.pop()
                        adjacency_matrix[j, i] = adjacency_matrix[i, j]
        else:
            raise ValueError("Model must be dumbbell, linear, or random.")
        return adjacency_matrix

    def show_adjacency_matrix(self):
        print(self.adjacency_matrix)
        
    def get_density(self):
        return np.sum(self.adjacency_matrix) / (self.n * (self.n - 1))
    
    def save_graph_with_labels(self, path):
        rows, cols = np.where(self.adjacency_matrix == 1)
        edges = zip(rows.tolist(), cols.tolist())
        G = nx.Graph()
        G.add_edges_from(edges)
        pos = nx.kamada_kawai_layout(G)
        nx.draw_networkx(G, pos=pos, with_labels=True)
        plt.savefig(path + '/adj_graph.png')


class MFRLEnv(gym.Env):
    actions = np.array([])
    def __init__(self, agent):
        self.adj_num = agent.get_adjacent_num()
        self.adj_ids = agent.get_adjacent_ids()
        self.adj_obs = {adj_id: [0, 0] for adj_id in self.adj_ids}
        self.counter = 0

        self.observation_space = spaces.Box(low=0, high=1, shape=(1, 2))
        self.action_space = spaces.Discrete(2)
        
    def reset(self, seed=None):
        super().reset(seed=seed)
        observation = np.array([0.5, 0.5])
        info = {}
        return observation, info
    
    def gather_actions(self, action):
        MFRLEnv.actions = np.append(MFRLEnv.actions, action)
        
    def calculate_meanfield(self):  
        # Return the meanfield observation
        if self.idle_check():
            return np.array([0, 1])
        else:
            return np.array(
                (self.adj_num*(2**self.adj_num)*np.array([0.5, 0.5]) - self.adj_num*np.array([1, 0])) 
                / (self.adj_num*(2**self.adj_num)-self.adj_num))

    def idle_check(self):
        # Check if all the adjacent agents are idle, based on MFRLEnv.actions
        if all(MFRLEnv.actions[self.adj_ids] == 0):
            return True
        else:
            return False
    
    def step(self, action):
        observation = self.calculate_meanfield()
        if action == 1 and self.idle_check():
            reward = 1
        else:
            reward = 0
        terminated = False
        self.counter += 1
        info = {}
        if self.counter == MAX_COUNTER:
            terminated = True
        return observation, reward, terminated, False, info


class Agent:
    def __init__(self, topology, id):
        self.topology = topology
        if id >= topology.n:
            raise ValueError("id must be less than n.")
        else:
            self.id = id
        self.env = MFRLEnv(self)
         
    def get_adjacent_ids(self):
        return np.where(self.topology.adjacency_matrix[self.id] == 1)[0]
    
    def get_adjacent_num(self):
        return len(self.get_adjacent_ids())

In [7]:
# Make topology
topology = Topology(12, "dumbbell", 0.5)
topology.show_adjacency_matrix()

# Make agents
MAX_COUNTER = 300
agents = [Agent(topology, i) for i in range(topology.n)]

[[0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0.]]


In [8]:
agents[0].topology.show_adjacency_matrix()

[[0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0.]]


In [16]:
a = [0]*10
print(a)
a[0] = np.array([1, 2])
print(a)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[array([1, 2]), 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [17]:
import pandas as pd
import numpy as np

df = pd.read_csv('agent_rewards.csv')

In [18]:
# Pivot the dataframe to have columns for each agent's reward at each step of each episode
df_pivot = df.pivot_table(index=['episode', 'step'], columns='agent_id', values='reward').reset_index()
# Rename the columns appropriately
df_pivot.columns = ['episode', 'step'] + [f'agent_{col}' for col in df_pivot.columns[2:]]
# Save the pivoted dataframe to a new CSV file
print(df_pivot)


      episode  step  agent_0  agent_1  agent_2  agent_3  agent_4
0           0     0        0        1        0        0        0
1           0     1        0        1        0        0        0
2           0     2        0        1        0        0        0
3           0     3        0        1        0        0        0
4           0     4        0        1        0        0        0
...       ...   ...      ...      ...      ...      ...      ...
2995        9   295        0        0        0        0        0
2996        9   296        0        0        0        0        0
2997        9   297        0        0        0        0        0
2998        9   298        0        0        0        0        0
2999        9   299        0        0        0        0        0

[3000 rows x 7 columns]


In [10]:
import torch

matrix = torch.arange(0,100).reshape(10,10)
indices = [0,2,4,6,8,0,2,4,6,8]
indices = torch.tensor(indices).unsqueeze(axis=-1)
print(matrix)
print(indices)
print(matrix.gather(0, indices))

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
        [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
        [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
        [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
        [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])
tensor([[0],
        [2],
        [4],
        [6],
        [8],
        [0],
        [2],
        [4],
        [6],
        [8]])
tensor([[ 0],
        [20],
        [40],
        [60],
        [80],
        [ 0],
        [20],
        [40],
        [60],
        [80]])


In [8]:
def outp1():
    return 1, 2

def outp2():
    return (1, 2)

print(outp1())
print(outp2())


(1, 2)
(1, 2)


In [4]:
import torch
import torch.nn as nn

m = nn.Softmax(dim=0)
input = torch.randn(2, 3)
output = m(input)
print(output)

tensor([[0.7532, 0.0908, 0.0377],
        [0.2468, 0.9092, 0.9623]])


In [2]:
import numpy as np

a = np.array([1, 2, 3])
b = 4

np.append(a, b)

array([1, 2, 3, 4])